In [1]:
from lib.workspace.zenodo_client import Zenodo
import os

from dotenv import load_dotenv
load_dotenv()

zenodo_datastore = Zenodo(os.getenv("ZENODO_TOKEN"))
published_depositions = zenodo_datastore.list_depositions()

deposition_ids = [file['id'] for file in published_depositions]
file_urls = [f'https://zenodo.org/api/records/{deposition_id}' for deposition_id in deposition_ids]
print(file_urls) 

['https://zenodo.org/api/records/14890740', 'https://zenodo.org/api/records/14881009', 'https://zenodo.org/api/records/14877594', 'https://zenodo.org/api/records/14874096', 'https://zenodo.org/api/records/14872284', 'https://zenodo.org/api/records/14871202', 'https://zenodo.org/api/records/14870313', 'https://zenodo.org/api/records/14869934', 'https://zenodo.org/api/records/14868704']


In [3]:
from datetime import date
import requests

# Iterate over each deposition_id in the file_urls list
file_links = []
for url in file_urls: 
    # Send a GET request to the URL
    response = requests.get(url).json()['files']
    link = "".join([file['links']['self'] for file in response])
    file_links.append(link)

print(file_links)

['https://zenodo.org/api/records/14890740/files/rent_contracts_2025-02-19.parquet/content', 'https://zenodo.org/api/records/14881009/files/rent_contracts_2025-02-17.parquet/content', 'https://zenodo.org/api/records/14877594/files/rent_contracts_2025-02-16.parquet/content', 'https://zenodo.org/api/records/14874096/files/rent_contracts_2025-02-15.parquet/content', 'https://zenodo.org/api/records/14872284/files/rent_contracts_2025-02-14.parquet/content', 'https://zenodo.org/api/records/14871202/files/rent_contracts_2025-02-14.parquet/content', 'https://zenodo.org/api/records/14870313/files/rent_contracts_2025-02-13.parquet/contenthttps://zenodo.org/api/records/14870313/files/rent_contracts_2025-02-14.parquet/content', 'https://zenodo.org/api/records/14869934/files/rent_contracts_2025-02-14.parquet/content', 'https://zenodo.org/api/records/14868704/files/rent_contracts_2025-02-13.parquet/content']


In [ ]:
# Download the file_links[0] using requests.get()
file_url = file_links[0]
response = requests.get(file_url)

# Save the content to a file
with open('output/rent_contracts.parquet', 'wb') as file:
    file.write(response.content)

In [ ]:
import polars as pl

lf = pl.scan_parquet('output/rent_contracts.parquet')

lf = lf.with_columns([
    pl.col("contract_start_date").str.strptime(pl.Date),
    pl.col("contract_end_date").str.strptime(pl.Date),
])

print(lf.collect_schema())

df = lf.collect()

print(df)